In [97]:
import pandas as pd
from IPython.display import JSON
import pylab
import json
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid') # TODO: what did this do?
import numpy as np
from IPython.core import display
import psycopg2
import os
homedir = os.getenv("HOME")
with open(homedir + "/.pgpass",'r') as pgfile:
    pgdat = pgfile.read().strip().split(":")
    pgurl = pgdat[0]
    pgport = pgdat[1]
    pgdb = pgdat[2]
    pguser = pgdat[3]
    pgpass = pgdat[4]

%load_ext sql
from sqlalchemy import create_engine
# sql info: https://github.com/catherinedevlin/ipython-sql
# on error `pip install ipython-sql`
#uncomment for remote
#pgport=3333
pgport=5433
engine = create_engine('postgresql://postgres@localhost:%i/postgres' % pgport)
db = psycopg2.connect(host=pgurl, database=pgdb, user=pguser,password=pgpass,port=pgport)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
def qry(q):
  return pd.read_sql_query(con = db, sql = q)
# pd.set_option("max_colwidth", None)
# pd.set_option('display.max_columns', None)
pd.set_option("display.latex.repr", False)
pd.set_option("display.latex.escape", True)

def iDisplay(v):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "max_colwidth", None):
        display.display(v)


Count number of uploaded APK files

In [70]:
%%sql postgresql://postgres@localhost:$pgport/postgres
select
    (select count(*) from apks where apkname not like 'jar_%' and apkname not like 'spec_%') as apk_count,
    (select count(*) from apks) as blob_count, 
    (select count(*) from inputs) as input_count, 
    (select count(*) from jobs) as job_count, 
    (select count(*) from resultdata) as result_data_count, 
    (select count(*) from results) as result_count

1 rows affected.


apk_count,blob_count,input_count,job_count,result_data_count,result_count
55,167,56,1410,927,952


In [85]:
%%sql -- count locations by pattern
select count(*), cast(jobs.config::json->'tag'->'heuristicType' as varchar) as category from jobs group by category

 * postgresql://postgres@localhost:5433/postgres
5 rows affected.


count,category
55,"""SensitiveDerefFieldCausedSynch"""
357,"""Disallow.I_CIEnter_Dialogdismiss"""
163,"""Disallow.I_CIEnter_AsyncTaskexecute"""
33,"""SensitiveDerefFieldCausedFinish"""
802,"""SensitiveDerefCallinCaused"""


In [89]:
%%sql
select count(*), status from jobs where cast(jobs.config::json->'tag'->'heuristicType' as varchar) like '%Synch%' group by status

 * postgresql://postgres@localhost:5433/postgres
3 rows affected.


count,status
53,completed
1,"failed: java.lang.AssertionError: assertion failed: existing results data nonempty, ResultDir(140448,/dev/shm/bounder_out_tmp,ExpTag(SensitiveDerefFieldCausedSynch,,))java.lang.AssertionError: assertion failed: existing results data nonempty, ResultDir(140448,/dev/shm/bounder_out_tmp,ExpTag(SensitiveDerefFieldCausedSynch,,)) at scala.Predef$.assert(Predef.scala:279) at edu.colorado.plv.bounder.ExperimentsDb.finishSuccess(Driver.scala:1142) at edu.colorado.plv.bounder.ExperimentsDb.$anonfun$processJob$1(Driver.scala:906) at edu.colorado.plv.bounder.ExperimentsDb.processJob(Driver.scala:932) at edu.colorado.plv.bounder.ExperimentsDb.loop(Driver.scala:837) at edu.colorado.plv.bounder.Driver$.expLoop(Driver.scala:281) at edu.colorado.plv.bounder.Driver$.runAction(Driver.scala:343) at edu.colorado.plv.bounder.Driver$.main(Driver.scala:268) at edu.colorado.plv.bounder.Driver.main(Driver.scala)"
1,failed: Non-zero exit code. StdErr: --- StdOut:java.library.path set to: /usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/libsubs fast: 1subs fast: 2subs fast: 3subs fast: 4subs fast: 5subs fast: 6subs fast: 7subs fast: 8subs fast: 9subs fast: 10subs fast: 11subs fast: 12subs fast: 13subs fast: 14subs fast: 15subs fast: 16subs fast: 17subs fast: 18subs fast: 19subs fast: 20subs fast: 21


In [88]:
%%sql 
select count(*) from jobs where jobs.status like '%existing results data nonempty%'


 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


count
3


Print jobs and statuses

In [90]:
%%sql
select 
    (select count(*) from jobs where status = 'completed') as Completed_Jobs,
    (select count(*) from jobs where status = 'new') as New_Jobs,
    (select count(*) from jobs where status = 'paused') as Paused_Jobs,
    (select count(*) from jobs) as Total_Jobs,
    (select count(*) from jobs where status = 'acquired') as Acquired_jobs,
    (select count(id) from jobs 
       where stderr like '%Exception%no active body present for method%') as Body_Exception,
    (select count(id) from jobs where stderr like '%an implementation is missing%') as unimplemented,
    (select count(id) from jobs where stderr like '%Exception%') as Exception,
    (select count(id) from jobs where stderr like '%no libz3java in java.library.path%') as path_exception,
    (select count(id) from jobs where stderr like '%Failed to parse reflective type reference%') as Refl_exception,
    (select count(id) from jobs where status like 'failed%' and status not like 'failed: Subprocess Timeout') as worker_failure,
    (select count(id) from jobs where status like 'failed: Subprocess Timeout') as subprocess_timeout,
    (select count(id) from jobs where stdout like '%empty pred location%') as possible_cg_unsound,
    (select count(id) from jobs where stderr like '%OutOfMemory%') as memory
 

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


completed_jobs,new_jobs,paused_jobs,total_jobs,acquired_jobs,body_exception,unimplemented,exception,path_exception,refl_exception,worker_failure,subprocess_timeout,possible_cg_unsound,memory
972,0,0,1410,9,0,0,0,0,0,24,405,0,0


In [5]:
%%sql
select owner,count(*) from jobs where status ='completed' group by owner;

 * postgresql://postgres@localhost:5433/postgres
4 rows affected.


owner,count
Linux Loaner 5.19.0-35-generic #36-Ubuntu SMP PREEMPT_DYNAMIC Fri Feb 3 18:36:56 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux,147
Linux Loki 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux,37
Linux historia 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux,16
Linux historia-baseline 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux,210


In [100]:
res = %sql select jobs.config, results.result from results inner join jobs on results.jobid=jobs.id where cast(jobs.config::json->'tag' as varchar) like '%execute%' and cast(jobs.config::json->'apkPath' as varchar) like '%ch.blinkenlights.android%'
print(res[0][1])
print(res[1][1])
print()
print(res[1][0])
print()
JSON([json.loads(res[0][0]),json.loads(res[1][0])])


 * postgresql://postgres@localhost:5433/postgres
2 rows affected.
{"summary":"\"Unreachable\"","maxPathCh":"\"UnknownCharacterization\"","depth":"{\"depth\":-1,\"ordDepth\":-1,\"cbDepth\":-1,\"status\":\"Unreachable\"}","wit":[[]]}
{"summary":"\"Witnessed\"","maxPathCh":"\"UnknownCharacterization\"","depth":"{\"depth\":121,\"ordDepth\":1,\"cbDepth\":1,\"status\":\"Witnessed\"}","wit":[[["witnessed","WITNESSED: [CB Inv] ch.blinkenlights.android.vanilla.AudioPickerActivity void onCreate(android.os.Bundle)\n       state: (    heap:           pure:          types: List()    trace: )"],["witnessed","WITNESSED: [CB Inv] ch.blinkenlights.android.vanilla.AudioPickerActivity void onCreate(android.os.Bundle)\n       state: (    heap:           pure:          types: List()    trace: )"]]]}

{"apkPath":"${baseDir}/fdroid/ch.blinkenlights.android.vanilla/1.2.2/apk/ch.blinkenlights.android.vanilla_12200.apk","outFolder":["${baseDirOut}/ch.blinkenlights.android.vanilla"],"specSet":"{\"specs\":[],\"di

<IPython.core.display.JSON object>

In [50]:
#-- TODO: figure out why other exit code is occuring
res = %sql select status from jobs where status like 'failed%' and status not like 'failed: Subprocess Timeout' and status not like 'failed: Non-zero exit%' order by random() limit 30;
print(res[0])

 * postgresql://postgres@localhost:5433/postgres
6 rows affected.
('failed: java.lang.IllegalStateException: Concurrency exception, I am Linux historia-baseline 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC ... (894 characters truncated) ... .runAction(Driver.scala:343)\n\tat edu.colorado.plv.bounder.Driver$.main(Driver.scala:268)\n\tat edu.colorado.plv.bounder.Driver.main(Driver.scala)\n',)


In [8]:
%%sql
select jobid,count(*) from results group by jobid order by count desc limit 7;

 * postgresql://postgres@localhost:5433/postgres
7 rows affected.


jobid,count
140087,6
139136,4
139898,2
140227,2
139359,2
140260,2
139354,2


In [9]:
%%sql
select id,count(*) from jobs group by id order by count(*) desc limit 5 

 * postgresql://postgres@localhost:5433/postgres
5 rows affected.


id,count
139649,1
140415,1
140012,1
139575,1
139780,1


In [10]:
qry = %sql select qry from results limit 1;
print(qry[0][0])

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.
{"methodName":"java.lang.Void doInBackground(java.lang.Void[])","line":21,"t":"ReceiverNonNull","matcher":"android.content.pm.PackageManager getPackageManager()","className":"org.sufficientlysecure.keychain.ui.DecryptListFragment$2"}


In [11]:
# find exceptions
#%%sql 
#select replace(cast(stderr as varchar),'$','\$') from jobs where stderr like '%Exception%' limit 1;
outs = %sql select stderr,config from jobs where stderr like '%Exception%' and stderr not like '%OutOfMemory%' order by random() limit 1;
for out in outs:
    print(out[1])
    print(out[0])


 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


In [12]:
%%sql
select cast(results.result::json->'summary' as varchar) as summary, cast(jobs.config::json->'apkPath' as varchar) as benchmark,count(*) from results inner join jobs on jobs.id=results.jobid group by summary,benchmark order by benchmark,summary desc

 * postgresql://postgres@localhost:5433/postgres
75 rows affected.


summary,benchmark,count
"""\""Timeout\""""","""${baseDir}/fdroid/be.digitalia.fosdem/2.1.1/apk/be.digitalia.fosdem_1900211.apk""",1
"""\""Witnessed\""""","""${baseDir}/fdroid/ch.blinkenlights.android.vanilla/1.2.2/apk/ch.blinkenlights.android.vanilla_12200.apk""",3
"""\""Unreachable\""""","""${baseDir}/fdroid/ch.blinkenlights.android.vanilla/1.2.2/apk/ch.blinkenlights.android.vanilla_12200.apk""",2
"""\""Timeout\""""","""${baseDir}/fdroid/ch.blinkenlights.android.vanilla/1.2.2/apk/ch.blinkenlights.android.vanilla_12200.apk""",2
"""\""Ijava.lang.AssertionError: assertion failed: Exception: Too many locations found: \\n \""""","""${baseDir}/fdroid/ch.blinkenlights.android.vanilla/1.2.2/apk/ch.blinkenlights.android.vanilla_12200.apk""",1
"""\""Witnessed\""""","""${baseDir}/fdroid/com.android.keepass/2.6.8/apk/com.android.keepass_215.apk""",2
"""\""Unreachable\""""","""${baseDir}/fdroid/com.android.keepass/2.6.8/apk/com.android.keepass_215.apk""",3
"""\""Witnessed\""""","""${baseDir}/fdroid/com.asksven.betterbatterystats/3.3.1/apk/com.asksven.betterbatterystats_107.apk""",8
"""\""Unreachable\""""","""${baseDir}/fdroid/com.asksven.betterbatterystats/3.3.1/apk/com.asksven.betterbatterystats_107.apk""",1
"""\""Witnessed\""""","""${baseDir}/fdroid/com.byagowi.persiancalendar/7.8.0/apk/com.byagowi.persiancalendar_780.apk""",1


In [13]:
# find failed job
failed = %sql select jobs.config,results.qry from jobs inner join results on jobs.id=results.jobid where cast(results.result::json->'summary' as varchar) like '%Unknown, no more axioms%'

if len(failed) > 0:
    print(failed[0][0])


 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


Get failing apks

In [14]:
%%sql
select cast(config::json->'apkPath' as text) from jobs 
        where stderr like '%Exception%' group by cast(config::json->'apkPath' as text) ;

 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


text


In [15]:
%%sql
select cast(config::json->'apkPath' as text) as app,count(*) as count from jobs group by app order by count desc;

 * postgresql://postgres@localhost:5433/postgres
48 rows affected.


app,count
"""${baseDir}/fdroid/net.osmand.plus/4.3.8/apk/net.osmand.plus_4308.apk""",285
"""${baseDir}/fdroid/org.sufficientlysecure.keychain/5.8.2/apk/org.sufficientlysecure.keychain_58902.apk""",142
"""${baseDir}/fdroid/org.mariotaku.twidere/4.1.8/apk/org.mariotaku.twidere_517.apk""",92
"""${baseDir}/fdroid/net.wigle.wigleandroid/2.63/apk/net.wigle.wigleandroid_263.apk""",88
"""${baseDir}/fdroid/dev.ukanth.ufirewall/3.5.3/apk/dev.ukanth.ufirewall_20220615.apk""",72
"""${baseDir}/fdroid/com.seafile.seadroid2/2.2.48/apk/com.seafile.seadroid2_123.apk""",63
"""${baseDir}/fdroid/org.liberty.android.fantastischmemo/10.11.7/apk/org.liberty.android.fantastischmemo_237.apk""",56
"""${baseDir}/fdroid/de.blau.android/18.1.1.0/apk/de.blau.android_2303.apk""",54
"""${baseDir}/fdroid/de.blinkt.openvpn/0.7.43/apk/de.blinkt.openvpn_198.apk""",44
"""${baseDir}/fdroid/com.ichi2.anki/2.15.6/apk/com.ichi2.anki_21506300.apk""",41


In [29]:
%%sql 
select cast(result::json->'summary' as varchar) as summary, cast(result::json->'maxPathCh' as varchar) as ch,count(*),avg(querytime),max(querytime) from results group by summary,ch order by summary,ch;

 * postgresql://postgres@localhost:5433/postgres
6 rows affected.


summary,ch,count,avg,max
"""\""Ijava.lang.AssertionError: assertion failed: Exception: Too many locations found: \\n \""""","""\""UnknownCharacterization\""""",1,10.0000000000000000,10
"""\""Ijava.lang.AssertionError: assertion failed: Exception: Too many locations found: \\n pre-line: 438 $r6 = virtualinvoke $r3.<androidx.fragment.app.FragmentActivity: java.lang.String toString()>()\\\\npre-line: 438 $r6 = virtualinvoke $r5.<java.lang.StringBuilder: java.lang.String toString()>()\""""","""\""UnknownCharacterization\""""",1,985.0000000000000000,985
"""\""Ijava.lang.IllegalArgumentException: cannot union primitive type set with: PrimTypeSet(byte)\""""","""\""UnknownCharacterization\""""",27,146.0000000000000000,520
"""\""Timeout\""""","""\""UnknownCharacterization\""""",48,791.6875000000000000,1762
"""\""Unreachable\""""","""\""UnknownCharacterization\""""",197,15.5228426395939086,566
"""\""Witnessed\""""","""\""UnknownCharacterization\""""",169,27.7278106508875740,705


In [18]:
%%sql 
select regexp_substr(cast(jobs.config::json->'apkPath' as varchar), 'AntennaPod|connectbot|sgtpuzzles') as benchmark, 
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute') as category, 
    count(*)
    from results inner join jobs on results.jobid=jobs.id 
    group by regexp_substr(cast(jobs.config::json->'apkPath' as varchar), 'AntennaPod|connectbot|sgtpuzzles'),
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute')

 * postgresql://postgres@localhost:5433/postgres
6 rows affected.


benchmark,category,count
None,None,253
sgtpuzzles,Dialog_dismiss,2
connectbot,execute,1
None,execute,61
None,Dialog_dismiss,109
connectbot,Dialog_dismiss,2


In [91]:
%%sql 
select count(*), cast(results.result::json->'summary' as varchar),cast(jobs.config::json->'tag' as varchar) as tag 
from jobs inner join results on jobs.id=results.jobid group by cast(jobs.config::json->'tag' as varchar),cast(results.result::json->'summary' as varchar)
order by  cast(jobs.config::json->'tag' as varchar)

 * postgresql://postgres@localhost:5433/postgres
20 rows affected.


count,varchar,tag
3,"""\""Ijava.lang.IllegalArgumentException: cannot union primitive type set with: PrimTypeSet(byte)\""""","{""heuristicType"":""Disallow.I_CIEnter_AsyncTaskexecute""}"
31,"""\""Timeout\""""","{""heuristicType"":""Disallow.I_CIEnter_AsyncTaskexecute""}"
30,"""\""Unreachable\""""","{""heuristicType"":""Disallow.I_CIEnter_AsyncTaskexecute""}"
68,"""\""Witnessed\""""","{""heuristicType"":""Disallow.I_CIEnter_AsyncTaskexecute""}"
33,"""\""Ijava.lang.IllegalArgumentException: cannot union primitive type set with: PrimTypeSet(byte)\""""","{""heuristicType"":""Disallow.I_CIEnter_Dialogdismiss""}"
34,"""\""Timeout\""""","{""heuristicType"":""Disallow.I_CIEnter_Dialogdismiss""}"
62,"""\""Unreachable\""""","{""heuristicType"":""Disallow.I_CIEnter_Dialogdismiss""}"
126,"""\""Witnessed\""""","{""heuristicType"":""Disallow.I_CIEnter_Dialogdismiss""}"
2,"""\""Ijava.lang.AssertionError: assertion failed: Exception: Too many locations found: \\n \""""","{""heuristicType"":""SensitiveDerefCallinCaused""}"
1,"""\""Ijava.lang.AssertionError: assertion failed: Exception: Too many locations found: \\n pre-line: 438 $r6 = virtualinvoke $r3.<androidx.fragment.app.FragmentActivity: java.lang.String toString()>()\\\\npre-line: 438 $r6 = virtualinvoke $r5.<java.lang.StringBuilder: java.lang.String toString()>()\""""","{""heuristicType"":""SensitiveDerefCallinCaused""}"


In [20]:
%%sql 
select regexp_substr(cast(jobs.config::json->'apkPath' as varchar), '[a-zA-Z]*\.[a-zA-Z]*') as benchmark, 
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute') as isDismiss, 
    cast(results.result::json->'summary' as varchar) as summary, count(*)
    from results inner join jobs on results.jobid=jobs.id 
    group by cast(jobs.config::json->'apkPath' as varchar),cast(results.result::json->'summary' as varchar),
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute')


 * postgresql://postgres@localhost:5433/postgres
122 rows affected.


benchmark,isdismiss,summary,count
com.android,execute,"""\""Witnessed\""""",1
org.liberty,execute,"""\""Witnessed\""""",9
org.navitproject,Dialog_dismiss,"""\""Witnessed\""""",2
com.darshancomputing,None,"""\""Unreachable\""""",7
com.ubergeek,Dialog_dismiss,"""\""Witnessed\""""",2
nl.mpcjanssen,None,"""\""Unreachable\""""",1
net.wigle,None,"""\""Witnessed\""""",17
dev.ukanth,Dialog_dismiss,"""\""Unreachable\""""",7
org.zephyrsoft,execute,"""\""Witnessed\""""",1
de.blinkt,Dialog_dismiss,"""\""Unreachable\""""",3


In [ ]:
%%sql
select cast(result::json->'summary' as varchar) as summary, count(*) from results, group by cast(result::json->'summary' as varchar)

In [ ]:
%%sql
select id from results where cast(results.result::json->'summary' as varchar) like '%Witnessed%'

In [ ]:
cfg = %sql select jobs.config from results inner join jobs on results.jobid=jobs.id where cast(results.result::json->'summary' as varchar) like '%Witnessed%' limit 1;
cfg[0][0]

In [ ]:
%%sql
select id,cast(result::json->'summary' as varchar) as summary,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results order by random() limit 5;

Check result consistency
========================

In [ ]:
%%sql -- get workers with acquired jobs -- check for abandoned jobs
select owner,count(*) from jobs where status='acquired' group by owner;

In [ ]:
%%sql -- Find completed jobs where no results exist and no error occurred
select jobs.id from jobs where jobs.id not in (select results.jobid from results) and jobs.status = 'completed' and jobs.stderr not like '%Exception%';

In [ ]:
%%sql -- Find new/paused jobs with results
select jobs.id from jobs where jobs.id in (select results.jobid from results) and (jobs.status = 'paused' or jobs.status = 'new');

In [ ]:
%%sql -- check that each location/qry pair only has one result (should detect duplicat runs)
select count(*),qry,loc as num from results group by qry,loc order by count(*) desc limit 1;

Triage witness results
----------------------

In [ ]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Witnessed%' order by id asc;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWit.csv')

In [ ]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Itimeout%' and cast(result::json->'maxPathCh' as varchar) like '%MultiCallback%' order by id asc;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWitMultiTimeout.csv')

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'line'->'str' as varchar),'$','\$') from results where id = 40;

Triage proven results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results 
                where cast(result::json->'summary' as varchar) like '%Proven%' order by random() limit 10;

Triage unreachable results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz 
            from results where cast(result::json->'summary' as varchar) like '%Unreachable%' 
            order by random() limit 15;

In [ ]:
%%sql
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'method'->>0 as varchar),'$','\$') as mname, 
        replace(cast(loc::json->'line' as varchar),'$','\$') as location 
        from results where id = 15037;

In [ ]:
%%sql
select id,result,replace(cast(loc::json->'line' as varchar),'$','\$') as location 
    from results where loc like '%244 $r3%';

In [ ]:
%%sql
select replace(cast(loc as varchar),'$','\$') from results where id = 1494;

Runtime Stats
=============

In [ ]:
queryTime = %sql select querytime/60/60 from results;
tplt = queryTime.DataFrame().plot.hist(bins=16)
tplt.set_yscale('log')
# tplt.set_xscale('log')
tplt.set_xlabel('time (hrs)')
# split this into proven and witnessed

In [ ]:
%%sql
select 

In [ ]:
cfg = %sql select jobs.config from results join jobs on jobs.id = results.jobid where querytime > 6*60*60 limit 1;
if len(cfg) > 0:
    print(cfg[0][0])

In [ ]:
%%sql -- which apps cause runtimes over 2 hours?
select jobs.id, results.querytime, jobs.config::json->'apkPath' from results inner join jobs on jobs.id=results.jobid where results.querytime > 7200 order by results.querytime desc limit 200;

In [ ]:
# Job runtimes
runtimes = %sql select ended - started as time from jobs where ended is not NULL;
runtimesDF = runtimes.DataFrame()
runtimesDF['time'] = runtimesDF['time'].apply(lambda x: x.total_seconds() / (60))
tplt = runtimesDF.plot.hist(bins=32)
tplt.set_yscale('log')
tplt.set_xlabel('time (min)')

In [ ]:
# Plot job time trend
rtvt = %sql select started, ended - started as time from jobs where ended is not NULL;
rtvtDF = rtvt.DataFrame()
rtvtDF.sort_values('started', ascending=True)
fig, ax = plt.subplots()
ax.plot_date(rtvtDF['started'], rtvtDF['time'], marker='*', linestyle='')

fig.autofmt_xdate()
plt.show()


Total job time

In [ ]:
%%sql
select sum(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select avg(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select avg(querytime) from results;

In [ ]:
%%sql
select now() - min(started) from jobs;

Size of postgres data
=====================

In [ ]:
%%sql
WITH RECURSIVE pg_inherit(inhrelid, inhparent) AS
    (select inhrelid, inhparent
    FROM pg_inherits
    UNION
    SELECT child.inhrelid, parent.inhparent
    FROM pg_inherit child, pg_inherits parent
    WHERE child.inhparent = parent.inhrelid),
pg_inherit_short AS (SELECT * FROM pg_inherit WHERE inhparent NOT IN (SELECT inhrelid FROM pg_inherit))
SELECT table_schema
    , TABLE_NAME
    , row_estimate
    , pg_size_pretty(total_bytes) AS total
    , pg_size_pretty(index_bytes) AS INDEX
    , pg_size_pretty(toast_bytes) AS toast
    , pg_size_pretty(table_bytes) AS TABLE
  FROM (
    SELECT *, total_bytes-index_bytes-COALESCE(toast_bytes,0) AS table_bytes
    FROM (
         SELECT c.oid
              , nspname AS table_schema
              , relname AS TABLE_NAME
              , SUM(c.reltuples) OVER (partition BY parent) AS row_estimate
              , SUM(pg_total_relation_size(c.oid)) OVER (partition BY parent) AS total_bytes
              , SUM(pg_indexes_size(c.oid)) OVER (partition BY parent) AS index_bytes
              , SUM(pg_total_relation_size(reltoastrelid)) OVER (partition BY parent) AS toast_bytes
              , parent
          FROM (
                SELECT pg_class.oid
                    , reltuples
                    , relname
                    , relnamespace
                    , pg_class.reltoastrelid
                    , COALESCE(inhparent, pg_class.oid) parent
                FROM pg_class
                    LEFT JOIN pg_inherit_short ON inhrelid = oid
                WHERE relkind IN ('r', 'p')
             ) c
             LEFT JOIN pg_namespace n ON n.oid = c.relnamespace
  ) a
  WHERE oid = parent
) a
ORDER BY total_bytes DESC limit 5;

Inspection of exceptions
========================

In [ ]:
%%sql
select id,config::json->'apkPath',replace(stderr,'$','\$') from jobs where stderr like '%Exception%';

In [ ]:
#count of total results
%sql select count(*) as Total_locations_computed from results